## Fonction to Raster
Prend en entrée une Satellite Image, un dossier et un nom et la sauve en JP2 dans le dossier considéré

In [6]:
! pip install pyarrow -q -q -q 
! pip install rasterio -q -q -q 
! pip install geopandas -q -q -q
!pip install matplotlib

In [9]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from plot_utils import *

import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
#import cv2
from PIL import Image as im

from datetime import date
import re
import pyproj
import os


In [19]:
path_s3_cayenne_data
path_local_cayenne_data

'/home/onyxia/work/detection-habitat-spontane/notebooks/../data/PLEIADES/2022/GUYANE'

In [11]:
update_storage_access()
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["PLEIADES"][2022]["guyane"])

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})



In [20]:
# DL PLEIADE martinique 2017
fs.download(
        rpath=f"{bucket}/{path_s3_cayenne_data}",
        lpath=f"{path_local_cayenne_data}",
        recursive=True)


In [111]:
def to_raster(satellite_image,directory_name,file_name):
    
    data = satellite_image.array
    crs  = satellite_image.crs
    transform = satellite_image.transform
    n_bands = satellite_image.n_bands

    metadata = {
        'dtype': data.dtype,
        'count': n_bands,
        'width': data.shape[2],
        'height': data.shape[1],
        'crs': crs,
        'transform': transform
    }

    if not os.path.exists(directory_name):
        os.mkdir(directory_name)

    # Save the array as a raster file in jp2 format
    with rasterio.open(directory_name + "/" + file_name, 'w', **metadata) as dst:
        dst.write(data, indexes = np.arange(n_bands)+1)


### Test sur une image

In [115]:
satellite_image = SatelliteImage.from_raster(
        file_path = f"{path_local_cayenne_data}"+ "/ORT_2022072050325085_0352_0545_U22N_16Bits.jp2",
        dep = None,
        date = None,
        n_bands= 4)

print(satellite_image.array.shape)
i = 2
directory_name = "../splitted_data"
file_name = "ORT_2022072050325085_0352_0545_U22N_16Bits"+"_"+str(i)+".tif"

to_raster(satellite_image,directory_name,file_name)



(4, 2000, 2000)


In [116]:
satell_image_read =  SatelliteImage.from_raster(
        file_path = directory_name + "/" +file_name,
        dep = None,
        date = None,
        n_bands= 4)

In [117]:
#satell_image_read.plot([3,1,2])
print(satell_image_read.array.shape)
print(satell_image_read.crs)
print(satell_image_read.transform)
print(satell_image_read.bounds)
print(satell_image_read.filename)

(4, 2000, 2000)
EPSG:2972
| 0.50, 0.00, 352000.00|
| 0.00,-0.50, 545000.00|
| 0.00, 0.00, 1.00|
BoundingBox(left=352000.0, bottom=544000.0, right=353000.0, top=545000.0)
ORT_2022072050325085_0352_0545_U22N_16Bits_2.tif


## Test sur l'ensemble des images

- je prends un répertoire en entrée Par exemple Guyane et je lis je split et réécris les images en taille 250
- compter le nombre d'images à traiter et le nombre d'images à l'arrivée.

In [134]:
from tqdm import tqdm
# params 
file_path = f"{path_local_cayenne_data}"
tile_size = 250
n_bands = 3
directory_name = "../splitted_data"

#  fonction
list_name = os.listdir(file_path)
list_path = [file_path + "/" + name for name in list_name]

for path, file_name in tqdm(zip(list_path, list_name)): # tqdm ici 
    
    big_satellite_image = SatelliteImage.from_raster(
        file_path = path,
        dep = None,
        date = None,
        n_bands= 3
    )
    
    list_satellite_image = big_satellite_image.split(tile_size)
    
for i, satellite_image in enumerate(list_satellite_image):

        file_name_i = file_name.split(".")[0]+"_"+str(i)+".tif"
        to_raster(satellite_image,directory_name,file_name_i)


    # ajouter nombre de fichier en entrée nombre de fichiers en sortie etc..
    
    



240it [00:26,  9.13it/s]
